In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [2]:
from pre_process_func import pre_process

In [3]:
import glob 
import numpy as np
from pathlib import Path

In [42]:
path2splits="/scratch/enis/data/nna/labeling/splits_real/"
output_dir="/scratch/enis/data/nna/labeling/splits_preprocessed_real/"

In [54]:
files=glob.glob(path2splits+"*")
files_done=glob.glob(output_dir+"*")
files =[Path(f) for f in files]
files_done =[Path(f) for f in files_done]


In [56]:
processingDict={Path(f).stem:False for f in files}

for a_filedone in files_done:
    a_filedone=Path(a_filedone).stem.replace("_preprocessed","")
    processingDict[a_filedone]=True

In [58]:
doneFilesCount=sum(processingDict.values())
leftFileCount=len(processingDict)-doneFilesCount
doneFilesCount,leftFileCount

(87, 10012)

In [59]:
# make sure all files are mp3
for f in left_files:
    suffix=f.split(".")[-1]
    if not (suffix=="flac" or suffix=="mp3"):
        print(f)

In [60]:
from IPython.display import display
import ipywidgets as widgets



In [61]:
# make sure all files are mp3
# sounds=np.empty((0,96,64))
try:
    progress = widgets.FloatProgress(value=0.0, min=0.0, max=len(processingDict)-doneFilesCount)
    display(progress)

    for i,(f) in enumerate(files):
        if processingDict[f.stem]==False:
            sound=pre_process(f,output_dir=output_dir, saveAsFile=True)
            progress.value+=1
            processingDict[f.stem]=True
        
    !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "pre-process succeded"  
except Exception as e: 
    print(e)
    !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "pre-process failed"   

FloatProgress(value=0.0, max=10012.0)

KeyboardInterrupt: 

In [11]:
files=glob.glob(output_dir+"*")
sounds=[]
for f in files:
    sound=np.load(f)
    sounds.append(sound)

sounds=np.concatenate(sounds)

In [12]:
sounds.shape

(232890, 96, 64)

In [13]:
np.save("pre_processedv2.npy",sounds)



In [14]:
soundv1=np.load("/home/enis/projects/unsupervised_audio/pre_processed.npy")

In [15]:
soundv1.shape

(104390, 96, 64)

In [16]:
soundcombined=np.concatenate([soundv1,sounds])

In [17]:
np.save("/home/enis/projects/unsupervised_audio/pre_processedv1_v2.npy",soundcombined)

In [7]:
sounds=np.load("/home/enis/projects/unsupervised_audio/pre_processed.npy")

In [12]:
small_part=sounds[:1000]

In [16]:
np.save("/home/enis/projects/unsupervised_audio/small_pre_processed.npy",small_part)

In [ ]:
# python train_model.py &>> job_logs/logs.txt; python /home/enis/projects/nna/src/slack_message.py -t "unsupervised ended" -f job_logs/logs.txt


In [20]:
!conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "split ended" -t "logs_training" -f "/home/enis/projects/unsupervised_audio/job_logs/logs.txt"

In [21]:
!ls /home/enis/projects/unsupervised_audio/job_logs/logs.txt

/home/enis/projects/unsupervised_audio/job_logs/logs.txt
